In [ ]:
!pip install transformers
!pip install miditok
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 6.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import transformers
import miditok
import numpy
import os
import pathlib
from tensorflow import keras
import json
from transformers import TFGPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, GenerationConfig
from miditok import OctupleMono, MIDITokenizer, MIDILike
from miditok.constants import CHORD_MAPS
from miditoolkit import MidiFile
import tqdm

In [ ]:
attn_window = 10
j = 0
input_seq = [] 
target_seq = []
batch_size = 16
buffer_size = 1000


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
directory = "/content/gdrive/MyDrive/dummy/tokens/"

for fname in os.listdir(directory):                     # for each file in the directory
    with open(os.path.join(directory, fname)) as i:     # open the file
        data = json.load(i)                             # load the json file 

    for i in range(0, len(data['ids'][0])-1, 11):    # for each token in the file
            x = data['ids'][0][i:i+attn_window]      # get the next 10 tokens
            y = data['ids'][0][i+1:i+attn_window+1]  # get the next 1+10 tokens

            if len(x) == attn_window and len(y) == attn_window: # if the length of the tokens is 10
                
                print('batch number:', j)            # print the batch number
                print('input:', x)                   
                print('target:', y) 
                print('-----')

                j += 1                               # increment the batch number

                input_seq.append(x)                  # append the input sequence
                target_seq.append(y)                 # append the target sequence

batch number: 0
input: [[49, 27, 14, 3, 3, 18], [54, 27, 14, 3, 3, 18], [57, 27, 14, 3, 3, 18], [49, 27, 14, 15, 3, 18], [53, 27, 14, 15, 3, 18], [57, 27, 14, 15, 3, 18], [50, 27, 18, 27, 3, 18], [54, 27, 18, 27, 3, 18], [57, 27, 18, 27, 3, 18], [47, 27, 26, 11, 4, 18]]
target: [[54, 27, 14, 3, 3, 18], [57, 27, 14, 3, 3, 18], [49, 27, 14, 15, 3, 18], [53, 27, 14, 15, 3, 18], [57, 27, 14, 15, 3, 18], [50, 27, 18, 27, 3, 18], [54, 27, 18, 27, 3, 18], [57, 27, 18, 27, 3, 18], [47, 27, 26, 11, 4, 18], [53, 27, 26, 11, 4, 18]]
-----
batch number: 1
input: [[56, 27, 26, 11, 4, 18], [54, 27, 14, 3, 5, 18], [57, 27, 14, 3, 5, 18], [61, 27, 14, 3, 5, 18], [53, 27, 14, 15, 5, 18], [57, 27, 14, 15, 5, 18], [61, 27, 14, 15, 5, 18], [54, 27, 18, 27, 5, 18], [57, 27, 18, 27, 5, 18], [62, 27, 18, 27, 5, 18]]
target: [[54, 27, 14, 3, 5, 18], [57, 27, 14, 3, 5, 18], [61, 27, 14, 3, 5, 18], [53, 27, 14, 15, 5, 18], [57, 27, 14, 15, 5, 18], [61, 27, 14, 15, 5, 18], [54, 27, 18, 27, 5, 18], [57, 27, 18, 2

In [ ]:
print('input_seq length:', len(input_seq))
print('target_seq length:', len(target_seq))

input_seq length: 24
target_seq length: 24


In [ ]:
from sklearn.model_selection import train_test_split

train_size = 0.8 # 80/20 split between train and test+val data
test_size = 0.5 # 50/50 (80/10/10) split between train, test and val

x_train, x_rem, y_train, y_rem = train_test_split(input_seq, target_seq, train_size = 0.8)
x_test, x_val, y_test, y_val = train_test_split(x_rem, y_rem, test_size = 0.5)


In [ ]:
print('train input:', len(x_train)), print('train target:', len(y_train))
print('val input:', len(x_val)), print('val target:', len(y_val))
print('test input:', len(x_test)), print('test target:', len(y_test))

train input: 19
train target: 19
val input: 3
val target: 3
test input: 2
test target: 2


(None, None)

In [ ]:

#creates tokenizer and loads in parameters
tokenizer = OctupleMono(params = '/content/gdrive/MyDrive/dummy/config.txt')

In [ ]:
# Creates model
config = GPT2Config( 
    vocab_size=1000,
    n_positions=attn_window,
    n_embd=64,
    n_layer=4,
    n_head=4,
    n_inner=128,
    resid_pdrop=.1,
    embd_pdrop=.1,
    attn_pdrop=.1,
    padding_token_id=tokenizer[0, 'PAD_None'],
    bos_token_id=tokenizer[0, 'BOS_None'],
    eos_token_id=tokenizer[0, 'EOS_None']
)
model = TFGPT2LMHeadModel(config)                      

In [ ]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])


In [ ]:
num_epoch = 100

# Create a callback that saves the model's weights
cp_callback = [

# tf.keras.callbacks.ModelCheckpoint("/content/gdrive/MyDrive/models_1/",
#                                                  verbose=1)

tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: model.save_pretrained("/content/gdrive/MyDrive/dummy/OCTMONO1/"))
]

history = model.fit(x_train, y_train, epochs=num_epoch, callbacks=[cp_callback], validation_data = (x_val, y_val))


Epoch 1/100
1/1 [==============================] - 22s 22s/step - loss: 6.6270 - accuracy: 0.2719 - val_loss: 6.5895 - val_accuracy: 0.4389
Epoch 2/100
1/1 [==============================] - 0s 209ms/step - loss: 6.6048 - accuracy: 0.3263 - val_loss: 6.5656 - val_accuracy: 0.5167
Epoch 3/100
1/1 [==============================] - 0s 174ms/step - loss: 6.5816 - accuracy: 0.4132 - val_loss: 6.5429 - val_accuracy: 0.5889
Epoch 4/100
1/1 [==============================] - 0s 174ms/step - loss: 6.5614 - accuracy: 0.4833 - val_loss: 6.5215 - val_accuracy: 0.6389
Epoch 5/100
1/1 [==============================] - 0s 177ms/step - loss: 6.5477 - accuracy: 0.5456 - val_loss: 6.5013 - val_accuracy: 0.6833
Epoch 6/100
1/1 [==============================] - 0s 185ms/step - loss: 6.5276 - accuracy: 0.5930 - val_loss: 6.4823 - val_accuracy: 0.6833
Epoch 7/100
1/1 [==============================] - 0s 166ms/step - loss: 6.5113 - accuracy: 0.6404 - val_loss: 6.4644 - val_accuracy: 0.6833
Epoch 8/100
1/